<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/model_build/Inception_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50

# **CallBack**

In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.70):
            print("\nReached 70.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [5]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

Cloning into 'skin-disease-build'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 18517 (delta 71), reused 106 (delta 32), pack-reused 18361
Receiving objects: 100% (18517/18517), 1.61 GiB | 22.63 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Checking out files: 100% (19583/19583), done.


In [6]:
train_dir = r"/content/skin-disease-build/dataset/train"
test_dir=r"/content/skin-disease-build/dataset/test"
os.listdir(train_dir)

['Melanoma Skin Cancer Nevi and Moles',
 'Seborrheic Keratoses and other Benign Tumors',
 'Lupus and other Connective Tissue diseases',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Light Diseases and Disorders of Pigmentation',
 'Systemic Disease',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Warts Molluscum and other Viral Infections',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Atopic Dermatitis Photos',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Nail Fungus and other Nail Disease',
 'Eczema Photos',
 'Urticaria Hives',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Bullous Disease Photos',
 'Herpes HPV and other STDs Photos',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Exanthems and Drug Eruptions',
 'Acne and Rosacea Photos',
 'Tinea Ringworm Candidiasis and other Fungal Infections']

# **Image Generator**

In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,   # randomly flip images
        validation_split = 0.2)

train_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'training')
valid_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'validation')


Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.


#**Model**

In [16]:
base_model = InceptionV3(input_shape = (200, 200, 3), include_top = False, weights = 'imagenet')
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(23, activation = 'softmax'))

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-25:]:
    layer.trainable = True

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer = optimizer , loss = "sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])
epochs = 40
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 4, 4, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32768)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 23)                2967      
Total params: 26,000,183
Trainable params: 7,017,879
Non-trainable params: 18,982,304
__________________________________

# **Compile and Run Model**

In [ ]:
history = model.fit_generator(train_gen,epochs = epochs,
                             validation_data = valid_gen,
                             steps_per_epoch = train_gen.n//train_gen.batch_size,
                             validation_steps = valid_gen.n//valid_gen.batch_size)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
389/389 [==============================] - 1293s 3s/step - loss: 3.4255 - sparse_categorical_accuracy: 0.0763 - val_loss: 3.0522 - val_sparse_categorical_accuracy: 0.0905
Epoch 2/40
389/389 [==============================] - 1299s 3s/step - loss: 3.0391 - sparse_categorical_accuracy: 0.0880 - val_loss: 3.0094 - val_sparse_categorical_accuracy: 0.0905
Epoch 3/40
389/389 [==============================] - 1297s 3s/step - loss: 3.0083 - sparse_categorical_accuracy: 0.0893 - val_loss: 2.9897 - val_sparse_categorical_accuracy: 0.0905
Epoch 4/40
211/389 [===============>..............] - ETA: 8:00 - loss: 2.9916 - sparse_categorical_accuracy: 0.0902

## **GRAPH**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

KeyError: ignored